In [1]:
# Imports
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

import state
import stateFactory
import nn_utils
from nn_utils import one_hot_encode

2023-03-10 11:25:22.736637: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 11:25:22.948152: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-10 11:25:22.983774: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-10 11:25:22.983789: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
quizzes = np.zeros((1000000, 81), np.int32)
solutions = np.zeros((1000000, 81), np.int32)

for i, line in enumerate(open('archive/sudoku.csv', 'r').read().splitlines()[1:]):
    quiz, solution = line.split(",")
    for j, q_s in enumerate(zip(quiz, solution)):
        q, s = q_s
        quizzes[i, j] = q
        solutions[i, j] = s

X = quizzes.reshape((-1, 9, 9))
Y = solutions.reshape((-1, 9, 9))

assert len(X) == len(Y)     # Check same number of X's and Y'x loaded

In [3]:
temp = one_hot_encode(Y[7])
print(temp.shape)
assert temp.shape == (9,9,9)

(9, 9, 9)


In [4]:
# Create a new Tensor to hold the One Hot Encoded puzzles
shape = one_hot_encode(X[0]).shape
X_ohe = np.ndarray((len(X), shape[0], shape[1], shape[2]))
Y_ohe = np.ndarray((len(Y), shape[0], shape[1], shape[2]))

In [5]:
delete_ids = []

for i in range(len(X)):
    try:
        X_ohe[i] = one_hot_encode(X[i])
        Y_ohe[i] = one_hot_encode(Y[i])
    except ValueError:
        print(f"{i} malformed")

14861 malformed
118801 malformed
147953 malformed
168906 malformed
228197 malformed
482410 malformed
760120 malformed
781414 malformed
865168 malformed
994456 malformed


In [6]:
# Training/Test split
X_train, X_test, Y_train, Y_test = train_test_split(X_ohe, Y_ohe, test_size=0.2, random_state=42)

print(f"Training X: {X_train.shape}")
print(f"Training Y: {Y_train.shape}")

Training X: (800000, 9, 9, 9)
Training Y: (800000, 9, 9, 9)


In [7]:
# Create model of the Neural Network

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv3D(512, kernel_size=(3,3,3), activation='relu', padding='same', input_shape=(9, 9, 9, 1)),
    
    tf.keras.layers.Conv3D(512, kernel_size=(3,3,3), activation='relu', padding='same'),  
    tf.keras.layers.Conv3D(512, kernel_size=(3,3,3), activation='relu', padding='same'),  
    tf.keras.layers.Conv3D(512, kernel_size=(3,3,3), activation='relu', padding='same'),  
    tf.keras.layers.Conv3D(512, kernel_size=(3,3,3), activation='relu', padding='same'),  
    tf.keras.layers.Conv3D(512, kernel_size=(3,3,3), activation='relu', padding='same'),  
    tf.keras.layers.Conv3D(512, kernel_size=(3,3,3), activation='relu', padding='same'),  
    tf.keras.layers.Conv3D(1, kernel_size=(3,3,3), activation='relu', padding='same'),  
])

2023-03-10 11:26:53.192671: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-10 11:26:53.192903: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-10 11:26:53.192927: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-03-10 11:26:53.193772: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# Compiling model

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 9, 9, 9, 512)      14336     
                                                                 
 conv3d_1 (Conv3D)           (None, 9, 9, 9, 512)      7078400   
                                                                 
 conv3d_2 (Conv3D)           (None, 9, 9, 9, 512)      7078400   
                                                                 
 conv3d_3 (Conv3D)           (None, 9, 9, 9, 512)      7078400   
                                                                 
 conv3d_4 (Conv3D)           (None, 9, 9, 9, 512)      7078400   
                                                                 
 conv3d_5 (Conv3D)           (None, 9, 9, 9, 512)      7078400   
                                                                 
 conv3d_6 (Conv3D)           (None, 9, 9, 9, 512)      7

In [ ]:
# Fit model to training data
model.fit(X_train, Y_train, batch_size=2, epochs=1)

    39/400000 [..............................] - ETA: 142:11:04 - loss: 13.2397 - accuracy: 0.1311

In [24]:
# Evaluate model
model.evaluate(X_test, Y_test)

6250/6250 [==============================] - 9s 1ms/step - loss: 0.3905 - accuracy: 0.0000e+00


[0.39053305983543396, 0.0]

In [25]:
tf.keras.models.save_model(
    model,
    "model/")

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [ ]:
# Predict results

test_sudoku = stateFactory.generate_valid_state()
test_sudoku_input = nn_utils.one_hot_encode(test_sudoku.board)

prediction = np.array(model.predict(test_sudoku_input)) # Create 1d-array solution
prediction_arr = prediction.reshape(9, 9, 9) # Put into 3 dimensions
result = nn_utils.get_max_index(prediction_arr) # Extract the max index

print(result)

In [19]:
### KEEP FOR LATER ###
# IOT return to sudoku shape use:
array = X_ohe[0]
np.reshape(array, (9,9,9))

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0